In [116]:
from langchain.document_loaders import TextLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.chains import VectorDBQA
from langchain.chains import RetrievalQA
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [117]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a chatbot having a conversation with a human."
        ),  # The persistent system prompt
        MessagesPlaceholder(
            variable_name="chat_history"
        ),  # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        ),  # Where the human input will injected
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

llm = ChatOpenAI(model='gpt-4')

# llm = OpenAI(model='gpt-4-vision-preview')

chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False,
    memory=memory,
)

# chat_llm_chain.predict(human_input="my name is angad")

In [118]:
# from langchain.schema.messages import HumanMessage, SystemMessage

# chat = ChatOpenAI(model="gpt-4-vision-preview")
# chat.invoke(
#     [
#         HumanMessage(
#             content=[
#                 {"type": "text", "text": """I'm giving you an image of a floorplan.
#                  """},
#                 {
#                     "type": "image_url",
#                     "image_url": {
#                         "url": "https://www.grundriss-schmiede.de/images/buerogrundriss/buerogrundriss.png",
#                         "detail": "auto",
#                     },
#                 },
#             ]
#         )
#     ]
# )

In [119]:
def split_string(input_string, max_length=180):
    # max_length = 125
    chunks = []

    current_chunk = ""
    for char in input_string:
        if char == '\n' or len(current_chunk) == max_length:
            chunks.append(current_chunk)
            current_chunk = ""
        else:
            current_chunk += char

    # Append the last chunk
    if current_chunk:
        chunks.append(current_chunk)

    return chunks

In [120]:
q_string = """Imagine an x-y cartesian grid with precision of 1 metre. Point A is at (100,100) and Point B is at (500,200).
What is the distance from point A to point B.
There are the following 3 rules to be obeyed during the journey.
Rule 1: YOU CANNOT CROSS THE LINE FROM (200,0) TO (200,200). THIS WALL HAS AN OPENING FROM (200,20) TO (200,30).
Rule 2: YOU CANNOT CROSS THE LINE FROM (200,200) TO (300,200).
Rule 3: YOU CANNOT CROSS THE LINE FROM (400,100) TO (400,300).
Rule 4: 0<x<600 & 0<y<300 for any point during the journey.
Rule 5: yOU CAN GO TO THE OTHER SIDE OF THE WALLS THROUGH THE OPENINGS IF MENTIONED. CHECK If going through them is the shorter path, do it.

AT EACH STEP, GO THROUGH ALL RULES TO CHECK IF EVERY RULE IS BEING OBEYED. EXPLAIN
THE ANSWER FOR EACH RULE. YOU CAN MOVE DIAGONALLY.
YOU CAN ALSO MOVE JUST BESIDE THE WALLS/OBSTACLES."""
# Keeping these rules in mind, tell if the distance to both point A and point B is more than 45 from any point
# in the cartesian plane""" 

# 3. There is an obstruction going from (200,0) to (200,200). So i can't go through this obstruction.

# q_string = """Give code to convert an image of a floorplan to a maze to go from one staircase
# to another in python."""

# q_string = """Given a floorplan with walls and two staircases, I want to convert it into an X-Y plane representation and generate a maze. 
# The walls in the floorplan should be translated into barriers in the maze, and the staircases should serve as entry and exit points.
#  Provide me with a detailed description or visualization of the resulting maze in the X-Y plane format.
# """

large_string = chat_llm_chain.predict(human_input=q_string)

# Split the string using the newline character
lines = split_string(large_string)

# Print each line
for line in lines:
    print(line)

Let's analyze the path and its compliance with the rules:

Step 1: Start at Point A (100,100). We're allowed to start here as it obeys Rule 4 (0<x<600 and 0<y<300).

Step 2: Move diagonally towards the opening in the wall at (200,20) - (200,30). This is the fastest path to the wall. The distance travelled is the Euclidean distance between (100,
00) and (200,25), which is approximately 111.8 meters. This route obeys Rule 1 (You cannot cross the line from (200,0) to (200,200) except the opening) and Rule 4.

Step 3: Cross through the opening from (200,25) to the other side of the wall. This is a vertical movement of 1 meter (from 25 to 26). This obeys Rule 1 and Rule 4.

Step 4: Move diagonally towards the point just below the next wall at (400,100). The Euclidean distance travelled is from (200,26) to (400,100), approximately 186.4 meters. This ob
ys Rule 3 (You cannot cross the line from (400,100) to (400,300)) and Rule 4.

Step 5: Move diagonally from (400,100) towards point B at (500

In [43]:
q_string = """give visual representation using _ | for walls and P for points of the room."""
# Provide me with a detailed description or visualization of the resulting maze in the X-Y plane format.
large_string = chat_llm_chain.predict(human_input=q_string)

# Split the string using the newline character
lines = split_string(large_string)

# Print each line
for line in lines:
    print(line)


I'm sorry for my previous response. As a text-based AI, I'm unable to create a detailed grid visual. However, I can provide a simple ASCII representation.

The grid would look something like this if you imagine each character as a 100 meter step:

```
P--------------------|          
|                    |          
|                    |          
|                    P          
|--------------------|          
|                              
|                              
P                              
```
In the above representation:

- P (Points): A is at the bottom left, B is at the top right.
- | (Walls): Represent the restrictions on the path.
- - (Path): The trajectory the path needs to follow.

Please keep in mind that this is a very simplified version of the grid and does not accurately represent the distances and proportions. For a more accurate visual representation, p
ease consider using graph paper or a digital drawing tool.


In [12]:
q_string = """can i give u .bxf or .dwg files"""

large_string = chat_llm_chain.predict(human_input=q_string)

# Split the string using the newline character
lines = split_string(large_string)

# Print each line
for line in lines:
    print(line)

I'm sorry for the confusion, but as a text-based AI, I don't have the ability to process files or view content like a human would. I can provide information and answer questions ba
ed on the text inputs I receive. If you have specific questions about your .bxf or .dwg files, feel free to ask and I'll do my best to help!
